# Inserting Data into Postgres Database

### Using pandas

In [ ]:
pip install SQLAlchemy

In [ ]:
pip install psycopg2

In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os, dotenv
from dotenv import load_dotenv
from datetime import datetime

In [2]:
load_dotenv()

database_name = os.getenv("DATABASE_NAME")
user = os.getenv("USER")
password = os.getenv("PASSWORD")
host = os.getenv("HOST")
port = os.getenv("PORT")

## Inserting data from Excel to Postgresql with pandas

### First Create Connction with postgress and create Table

1. Install Postgres

![sql_1](./sql/install.png)

2. Create Connection

![sql_2](./sql/connect.png)

3. Create sql file:
    *   Create database: upwork_1;
    *   Create all tables.

![sql](./sql/sql_data.png)

### Second Use pandas to access all sheets and insert data into postgresql database

In [3]:
#Define root
root = "../../excel/clean/data_cleaned.xlsx"
# Connect to PostgreSQL
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database_name}')

In [4]:
try:
    # Read Excel
    user = pd.read_excel(root, sheet_name="users", parse_dates=['birthday', 'created'])
    countries = pd.read_excel(root, sheet_name="countries")
    companies = pd.read_excel(root, sheet_name="companies")
    bills = pd.read_excel(root, sheet_name="bills",parse_dates=['created'])
    bills_items = pd.read_excel(root, sheet_name="bill_items")
    products = pd.read_excel(root, sheet_name="products",parse_dates=['release'])
    products_items = pd.read_excel(root, sheet_name="product_items")
    payments = pd.read_excel(root, sheet_name="payments",parse_dates=['created'])
    payment_methods = pd.read_excel(root, sheet_name="payment_methods")


    print("Data correctly obtained from excel")
except Exception as e:
    print(f"Error: {e}")

Data correctly obtained from excel


### Convert dates --> user['created']

In [5]:
def convert_dates(data,):
    new_list = []
    for i in range(len(data)):
        if len(data[i]) >= 19:
            new_list.append(data[i].split(' ')[0])
        else:
            new_list.append(datetime.strptime(data[i], '%d/%m/%Y').strftime('%Y-%m-%d'))
        
    return new_list

### Creating Objects

In [6]:
#Users
user_data = {
    "id": list(user["id"]),
    "username": list(user["username"]),
    "first_name": list(user["first_name"]),
    "last_name": list(user["last_name"]),
    "email": list(user["email"]),
    "password": list(user["password"]),
    "gender": list(user["gender"]),
    "birthday": [x.strftime('%Y-%m-%d') if x else None for x in user["birthday"]],
    "country_id": list(user["country_id"]),
    "created": convert_dates(user['created'].tolist())
}

#bills
bills_data = {
    "id": list(bills["id"]),
    "user_id": list(bills["user_id"]),
    "bill_address": list(bills["bill_address"]),
    "shipping_address": list(bills["shipping_address"]),
    "status": list(bills["status"]),
    "created": [x.strftime('%Y-%m-%d') if x else None for x in bills["created"]]
}

#Products
products_data = {
    "id": list(products["id"]),
    "product_name": list(products["product_name"]),
    "description": list(products["description"]),
    "price": list(products["price"]),
    "release": [x.strftime('%Y-%m-%d') if x else None for x in products["release"]],
    "department": list(products["department"])
}

#Payment
payment_data = {
    "id": list(payments["id"]),
    "payment_methods_id": list(payments["pay_methods_id"]),
    "bills_id": list(payments["bills_id"]),
    "transaction_id": list(payments["transaction_id"]),
    "status": list(payments["status"]),
    "created": [x.strftime('%Y-%m-%d') if x else None for x in payments["created"]]
}

In [7]:
new_users = pd.DataFrame(user_data)
new_bills = pd.DataFrame(bills_data)
new_products = pd.DataFrame(products_data)
new_payments = pd.DataFrame(payment_data)

In [ ]:
try:
    # Insert data
    countries.to_sql('countries', engine, if_exists='append', index=False)
    new_users.to_sql('users', engine, if_exists='append', index=False)
    companies.to_sql('companies', engine, if_exists='append', index=False)
    new_bills.to_sql('bills', engine, if_exists='append', index=False)
    bills_items.to_sql('bills_items', engine, if_exists='append', index=False)
    new_products.to_sql('products', engine, if_exists='append', index=False)
    products_items.to_sql('products_items', engine, if_exists='append', index=False)
    payment_methods.to_sql('payment_methods', engine, if_exists='append', index=False)
    new_payments.to_sql('payments', engine, if_exists='append', index=False)


    print("Data successfully imported to PostgreSQL!")
except Exception as e:
    print(f"Error: {e}")

Data successfully imported to PostgreSQL!
